In [15]:
%%writefile simd_divergence.c
#include <immintrin.h>
#include <stdio.h>

int main() {
    // Create a vector of integers
    __m256i values = _mm256_setr_epi32(0, 1, 2, 3, 4, 5, 6, 7);


    // Create mask: first 4 elements = true, last 4 = false
    __m256i mask = _mm256_setr_epi32(-1, -1, -1, -1, 0, 0, 0, 0);



    __m256i path_a = _mm256_slli_epi32(values, 1);  // * 2 (shift left)
    __m256i path_b = _mm256_mullo_epi32(values, _mm256_set1_epi32(3)); // * 3

    // Blend: if mask is true, take path_a, else path_b
    __m256i result = _mm256_blendv_epi8(path_b, path_a, mask);

    int out[8];
    _mm256_storeu_si256((__m256i*)out, result);

    for (int i = 0; i < 8; i++) { printf("%d | %d\n", i, out[i]); }

    return 0;
}

Overwriting simd_divergence.c


In [16]:
!gcc -O3 -Wall -Wextra -mavx2  -o simd_divergence simd_divergence.c
!./simd_divergence

0 | 0
1 | 2
2 | 4
3 | 6
4 | 12
5 | 15
6 | 18
7 | 21


In [17]:
# __global__ void warpDivergeExample() {
#    int tid = threadIdx.x;
#
#   // This if-else causes warp divergence
#   if (tid < 16) {
#       // First 16 threads (0-15) execute Path A
#       tid = tid * 2;
#   } else {
#       // Last 16 threads (16-31) execute Path B
#       tid = tid * 3;
#   }
#}